In [77]:
import os
import requests
import json
from dotenv import load_dotenv
import pandas as pd
import geopandas as gpd
from cartoframes.viz import Map, Layer, popup_element
from pymongo import MongoClient
from pymongo import GEOSPHERE
import math
import numpy as np

In [78]:
load_dotenv()

True

In [79]:
key = os.getenv("KEY")

In [80]:
client = MongoClient("localhost:27017")
db = client.get_database("Ironhack")

In [81]:
companies = db.get_collection("companies")

In [7]:
valencia_companies = list(companies.find({"offices.city":"Valencia"}))
barcelona_companies = list(companies.find({"offices.city":"Barcelona"}))
madrid_companies = list(companies.find({"offices.city":"Madrid"}))
spain_companies = [valencia_companies, barcelona_companies, madrid_companies]

In [8]:
name = []
city = []
office_number = []
address = []
latitude = []
longitude = []
for each_city in spain_companies:
    for company in each_city:
        try:
            for i in range(len(company["offices"])):
                n = 0
                if company["offices"][i]["city"] in ["Valencia", "Barcelona", "Madrid"]:
                    n += 1
                    name.append(company["name"])
                    city.append(company["offices"][i]["city"])
                    office_number.append(n)
                    address.append(company["offices"][i]["address1"])
                    latitude.append(company["offices"][i]["latitude"])
                    longitude.append(company["offices"][i]["longitude"])
        except:
            pass

In [9]:
spain_dict = {"name":name, "city":city, "office_number":office_number, "address":address, "latitude":latitude, "longitude":longitude}

In [10]:
df = pd.DataFrame.from_dict(spain_dict)

In [148]:
df['address'].replace('', np.nan, inplace=True)

In [152]:
df = df.dropna(subset=["address", "latitude", "longitude"], how='all')

In [161]:
def get_coordinates (place):
    try:
        res = requests.get(f"https://geocode.xyz/{place}?json=1").json()
        return [res["longt"],res["latt"]]
    except:
        return [np.nan, np.nan]

In [ ]:
for i in df.index.values.tolist():
    if math.isnan(df.at[i, "latitude"]) or df.at[i, "latitude"]==0:
        address_complete = df.at[i, "address"] + ", " + df.at[i, "city"] + ", Spain"
        coordinates = get_coordinates(address_complete)
        df.at[i, "latitude"]= float(coordinates[1])
        df.at[i, "longitude"]= float(coordinates[0])

In [196]:
for i in df.index.values.tolist():
    if df.at[i, "latitude"] < 20:
        latitude = df.at[i, "latitude"]
        longitude = df.at[i, "longitude"]
        df.at[i, "latitude"] = longitude
        df.at[i, "longitude"] = latitude

In [198]:
df['latitude'] = df['latitude'].fillna(0)

C:\Users\Daniel\AppData\Local\Temp\ipykernel_8060\3028638243.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['latitude'] = df['latitude'].fillna(0)


In [200]:
df.drop(df[df.latitude < 30].index, inplace=True)

C:\Users\Daniel\AppData\Local\Temp\ipykernel_8060\1358260333.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(df[df.latitude < 30].index, inplace=True)


In [209]:
df.drop(df[df.longitude < -10].index, inplace=True)

In [210]:
df.to_csv("Output/spanish_companies.csv")

In [6]:
df = pd.read_csv('Output/spanish_companies.csv')

In [7]:
df = df.set_index('Unnamed: 0')
df.index.name = None

In [8]:
position = ["Designers", "UI/UX Engineers", "Frontend Developers", "Data Engineers", 
             "Backend Developers", "Account Managers", "Maintenance", "Executives", "CEO"]
number_of_employees = [20, 5, 10, 15, 5, 20, 1, 10, 1]
salary = [30000, 33000, 33000, 44000, 39000, 25000, 20000, 60000, 80000]

In [9]:
employees_dict = {"position":position, "number_of_employees":number_of_employees, "salary":salary}

In [10]:
df2 = pd.DataFrame.from_dict(employees_dict)

In [11]:
df2

,position,number_of_employees,salary
0,Designers,20,30000
1,UI/UX Engineers,5,33000
2,Frontend Developers,10,33000
3,Data Engineers,15,44000
4,Backend Developers,5,39000
5,Account Managers,20,25000
6,Maintenance,1,20000
7,Executives,10,60000
8,CEO,1,80000


In [12]:
requirements = ["Design company", "Schools", "Successful tech startup", "Starbucks", "Airport", "Party", 
                "Vegan restaurant", "Basketball stadium", "Dog hairdresser"]
related_employees = ["Designers", "30% of All", "Frontend Developers/Backend Developers", "Executives", "Account Managers", "All", 
                     "CEO", "Maintenance", "All"]
weight = [80, 100, 80, 40, 100, 10, 300, 500, 1]

In [13]:
requirements_dict = {"requirements":requirements, "related_employees":related_employees, "weight":weight}

In [14]:
df3 = pd.DataFrame.from_dict(requirements_dict)

In [15]:
df3

,requirements,related_employees,weight
0,Design company,Designers,80
1,Schools,30% of All,100
2,Successful tech startup,Frontend Developers/Backend Developers,80
3,Starbucks,Executives,40
4,Airport,Account Managers,100
5,Party,All,10
6,Vegan restaurant,CEO,300
7,Basketball stadium,Maintenance,500
8,Dog hairdresser,All,1


In [16]:
final_weight = []
everyone = list(df2["position"].values)
related_employees = [["Designers"], everyone, ['Frontend Developers', 'Backend Developers'], ['Executives'], ['Account Managers'], everyone, ['CEO'], ['Maintenance'], everyone] 
for i, employee_types in enumerate(related_employees):
    weight = df3.at[i, "weight"]
    final = 0
    for employee_type in employee_types:
        number_of_employees = list(df2.loc[df2['position'] == employee_type]["number_of_employees"].values)[0]
        salary = list(df2.loc[df2['position'] == employee_type]["salary"].values)[0]
        final += weight*number_of_employees*salary
    if i == 1: 
        final *= 0.3
    final_weight.append(int(final))

In [18]:
df3['final_weight'] = final_weight

In [19]:
df3

,requirements,related_employees,weight,final_weight
0,Design company,Designers,80,48000000
1,Schools,30% of All,100,94500000
2,Successful tech startup,Frontend Developers/Backend Developers,80,42000000
3,Starbucks,Executives,40,24000000
4,Airport,Account Managers,100,50000000
5,Party,All,10,31500000
6,Vegan restaurant,CEO,300,24000000
7,Basketball stadium,Maintenance,500,10000000
8,Dog hairdresser,All,1,3150000


In [20]:
df

,name,city,office_number,address,latitude,longitude
0,e-contratos,Valencia,1,"Periodista Azzati, 5. 3, 6B",39.469301,-0.377068
1,Modo Marketing,Valencia,1,NaN,39.640730,-0.421944
2,Efebege,Valencia,1,"General Barroso, 66",39.450211,-0.392465
3,Feasa Valencia,Valencia,1,Cami Vera SN,37.210300,-1.857610
4,DOMODESK,Valencia,1,C/Alqueria de Benlloch 24,39.461268,-0.398346
...,...,...,...,...,...,...
159,cost4travel,Madrid,1,"Calle Francisco Jose Arroyo, 6",40.452871,-3.614939
160,Genasys,Madrid,1,C/ Ventura de la Vega 5,40.416184,-3.698873
162,FRV,Madrid,1,Rafael Calvo 42,40.433930,-3.690270
164,AdLINK Group,Madrid,1,affilinet Espana SLU,40.423070,-3.711930


In [37]:
def get_results_from_foursquare (query, location, limit):
    url = f"https://api.foursquare.com/v3/places/search?query={query}&ll={location[0]}%2C{location[1]}&radius=100000&sort=RELEVANCE&limit={limit}"
    headers = {
        "Accept": "application/json",
        "Authorization": key
    }
    response = requests.get(url, headers=headers)
    return response.json()["results"]

In [22]:
query = "school"
limit = 10
school_distance = []
for index in df.index.values.tolist():
    location = [df.at[index, "latitude"], df.at[index, "longitude"]]
    results = get_results_from_foursquare (query, location, limit)
    distances = []
    for result in results:
        distances.append(result["distance"])
    if len(distances)>0:
        starbucks_distance.append(min(distances))
    else:
        starbucks_distance.append(100000)
df['school_distance'] = school_distance

In [24]:
df.to_csv("Output/spanish_companies_school.csv")

In [33]:
query = "starbucks"
limit = 10
starbucks_distance = []
for index in df.index.values.tolist():
    location = [df.at[index, "latitude"], df.at[index, "longitude"]]
    results = get_results_from_foursquare (query, location, limit)
    distances = []
    for result in results:
        distances.append(result["distance"])
    if len(distances)>0:
        starbucks_distance.append(min(distances))
    else:
        starbucks_distance.append(100000)
df['starbucks_distance'] = starbucks_distance

In [35]:
df.to_csv("Output/spanish_companies_school_starbucks.csv")

In [39]:
query = "airport"
limit = 10
airport_distance = []
for index in df.index.values.tolist():
    location = [df.at[index, "latitude"], df.at[index, "longitude"]]
    results = get_results_from_foursquare (query, location, limit)
    distances = []
    for result in results:
        distances.append(result["distance"])
    if len(distances)>0:
        airport_distance.append(min(distances))
    else:
        airport_distance.append(100000)
df['airport_distance'] = airport_distance

In [41]:
df.to_csv("Output/spanish_companies_school_starbucks_airport.csv")

In [42]:
query = "vegan"
limit = 10
vegan_distance = []
for index in df.index.values.tolist():
    location = [df.at[index, "latitude"], df.at[index, "longitude"]]
    results = get_results_from_foursquare (query, location, limit)
    distances = []
    for result in results:
        distances.append(result["distance"])
    if len(distances)>0:
        vegan_distance.append(min(distances))
    else:
        vegan_distance.append(100000)
df['vegan_distance'] = vegan_distance

In [45]:
df.to_csv("Output/spanish_companies_school_starbucks_airport_vegan.csv")

In [48]:
query = "Basketball stadium"
limit = 10
basketball_distance = []
for index in df.index.values.tolist():
    location = [df.at[index, "latitude"], df.at[index, "longitude"]]
    results = get_results_from_foursquare (query, location, limit)
    distances = []
    for result in results:
        distances.append(result["distance"])
    if len(distances)>0:
        basketball_distance.append(min(distances))
    else:
        basketball_distance.append(100000)
df['basketball_distance'] = basketball_distance

In [50]:
df.to_csv("Output/spanish_companies_school_starbucks_airport_vegan_basketball.csv")

In [51]:
query = "party"
limit = 10
party_distance = []
for index in df.index.values.tolist():
    location = [df.at[index, "latitude"], df.at[index, "longitude"]]
    results = get_results_from_foursquare (query, location, limit)
    distances = []
    for result in results:
        distances.append(result["distance"])
    if len(distances)>0:
        party_distance.append(min(distances))
    else:
        party_distance.append(100000)
df['party_distance'] = party_distance

In [53]:
df.to_csv("Output/spanish_companies_school_starbucks_airport_vegan_basketball_party.csv")

In [68]:
query = "pet service"
limit = 10
dog_distance = []
for index in df.index.values.tolist():
    location = [df.at[index, "latitude"], df.at[index, "longitude"]]
    results = get_results_from_foursquare (query, location, limit)
    distances = []
    for result in results:
        distances.append(result["distance"])
    if len(distances)>0:
        dog_distance.append(min(distances))
    else:
        dog_distance.append(100000)
df['dog_distance'] = dog_distance

In [70]:
df.to_csv("Output/spanish_companies_school_starbucks_airport_vegan_basketball_party_dog.csv")

In [85]:
query = "Design"
limit = 10
design_distance = []
for index in df.index.values.tolist():
    location = [df.at[index, "latitude"], df.at[index, "longitude"]]
    results = get_results_from_foursquare (query, location, limit)
    distances = []
    for result in results:
        distances.append(result["distance"])
    if len(distances)>0:
        design_distance.append(min(distances))
    else:
        design_distance.append(100000)
df['design_distance'] = design_distance

In [87]:
df.to_csv("Output/spanish_companies_school_starbucks_airport_vegan_basketball_party_dog_design.csv")

In [88]:
query = "tech startup"
limit = 10
tech_distance = []
for index in df.index.values.tolist():
    location = [df.at[index, "latitude"], df.at[index, "longitude"]]
    results = get_results_from_foursquare (query, location, limit)
    distances = []
    for result in results:
        distances.append(result["distance"])
    if len(distances)>0:
        tech_distance.append(min(distances))
    else:
        tech_distance.append(100000)
df['tech_distance'] = tech_distance

In [90]:
df.to_csv("Output/spanish_companies_all.csv")

In [125]:
cols = df.columns.tolist()
cols2 = cols[0:6] + [cols[-3]] + [cols[6]] + [cols[-2]] + cols[7:9] + [cols[-5]] + cols[9:11] + [cols[-4]]
df = df[cols2]

In [128]:
df.to_csv("Output/spanish_companies_ordered.csv")

In [130]:
sum_of_distances = []
list_of_columns_to_add = list(df.columns)[6:]
for index in df.index.values.tolist():
    sum_ = 0
    i = 0
    for column in list_of_columns_to_add:
        weight = df3.at[i, "final_weight"]
        sum_ += df.at[index, column]*weight
        i += 1
    sum_of_distances.append(sum_)
df['sum_of_distances'] = sum_of_distances

In [138]:
df.to_csv("Output/spanish_companies_sum_of_distances.csv")

In [141]:
df["sum_of_distances"].idxmin()

0

In [149]:
df["sum_of_distances"][df['city']=="Valencia"].idxmin()

0

In [145]:
df["sum_of_distances"][df['city']=="Barcelona"].idxmin()

12

In [143]:
df["sum_of_distances"][df['city']=="Madrid"].idxmin()

164